# End-to-End Runbook

This notebook runs the full GridPulse workflow: data → features → splits → training → API smoke test.

In [ ]:
from pathlib import Path
import sys
repo_root = Path.cwd().parent if Path.cwd().name == 'notebooks' else Path.cwd()
print('Python:', sys.executable)
print('repo_root:', repo_root)

## 1) Install local package (editable)

In [ ]:
import subprocess
subprocess.run([str(repo_root / '.venv' / 'bin' / 'python'), '-m', 'pip', 'install', '-e', str(repo_root)], check=False)

## 2) Run data pipeline

In [ ]:
import subprocess
subprocess.run(['python', '-m', 'gridpulse.data_pipeline.validate_schema', '--in', str(repo_root/'data'/'raw'), '--report', str(repo_root/'reports'/'data_quality_report.md')], check=False)
subprocess.run(['python', '-m', 'gridpulse.data_pipeline.build_features', '--in', str(repo_root/'data'/'raw'), '--out', str(repo_root/'data'/'processed')], check=False)
subprocess.run(['python', '-m', 'gridpulse.data_pipeline.split_time_series', '--in', str(repo_root/'data'/'processed'/'features.parquet'), '--out', str(repo_root/'data'/'processed'/'splits')], check=False)

## 3) Train forecasting models

In [ ]:
import subprocess
subprocess.run(['python', '-m', 'gridpulse.forecasting.train', '--config', str(repo_root/'configs'/'train_forecast.yaml')], check=False)

## 4) Quick API smoke test

Start API in a terminal:

```bash
uvicorn services.api.main:app --reload --port 8000
```
Then run the checks below.

In [ ]:
import requests
base = 'http://localhost:8000'
print(requests.get(f'{base}/health').json())

In [ ]:
import requests
base = 'http://localhost:8000'
print(requests.get(f'{base}/forecast').json().get('meta', {}))

In [ ]:
import requests
base = 'http://localhost:8000'
payload = {
    'forecast_load_mw': [8000, 8200, 8100],
    'forecast_renewables_mw': [3200, 3100, 3300]
}
print(requests.post(f'{base}/optimize', json=payload).json())

## 5) Start the dashboard

```bash
streamlit run services/dashboard/app.py
```